In [1]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import rioxarray
import re
import rtree
import shapely
import pickle
import pyproj

from shapely.geometry import Polygon, Point
from shapely.ops import transform
#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook
import tqdm
# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

import data_eng.az_proc as ap
import data_eng.form_calcs as fc


#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

## File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

#create folder to hold tiles in completed dataset
#load tiles and tile xmls
tiles_dir = os.path.join(parent_directory,"complete_dataset","tiles")
os.makedirs(tiles_dir, exist_ok=True)

tiles_xml_dir = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_xml_dir, exist_ok=True)
tiles_xml_list = os.listdir(tiles_xml_dir)

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")

folders_of_images_xmls_by_tile = os.listdir(rechipped_image_path)
images_and_xmls_by_tile_path = rechipped_image_path

# Add chips to rechip folder

In [3]:
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
#    add_chips_to_chip_folders(rechipped_image_path, tile_name)

# Correct object labels

## Fix inconsistent labels 

for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    #specify image/xml paths for each tile
    positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
    fc.correct_inconsistent_labels_xml(positive_xml_dir)

## Reclassify narrow closed roof tanks and closed roof tanks by size

for tile in tqdm.tqdm(os.listdir(rechipped_image_path)): #iterate over tile folders
    tile_annotation_path = os.path.join(rechipped_image_path, tile, "chips_positive_xml")
    for xml in os.listdir(tile_annotation_path):
        xml_path = os.path.join(tile_annotation_path, xml)
        fc.reclassify_narrow_closed_roof_and_closed_roof_tanks(xml_path)

# Generate table of characteristics for tiles/images

In [5]:
#tile_characteristics, image_characteristics = fc.image_tile_characteristics(rechipped_image_path, tiles_dir)

In [3]:
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_characteristics = pd.read_csv("tile_characteristics.csv")

# Generate tile XMLs

In [4]:
#fc.generate_tile_xmls(rechipped_image_path, tiles_dir, tiles_xml_dir, int(512))

## Combine neighboring bounding boxes

https://answers.opencv.org/question/231263/merging-nearby-rectanglesedited/
https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one

In [4]:
distance_limit = 5
tile_database = fc.merge_tile_annotations(tile_characteristics, tiles_xml_dir, distance_limit = 5)
#merged_tile_database =  tile_database[tile_database["merged_bbox"] == True]

100%|██████████| 2120/2120 [59:53<00:00,  1.69s/it]  


# Clip storm surge data and append to tile level annotation database

In [5]:
sc_slosh_inundation_map_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/Natural Hazards Data/NHC Potential Storm Surge Flood Map/SC_SLOSH_MOM_Inundation"
tile_database = fc.identify_inundation_for_tanks(tile_database, sc_slosh_inundation_map_path)

# Add in state

In [6]:
states_gpds_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/US Political Boundaries/US State Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
tile_database = fc.identify_state_name_for_each_state(states_gpds_path, tile_database)
#check issues in state list
#state_list = state_list[state_list!=None]
#np.unique(state_list)

98169it [03:07, 522.35it/s] 


In [12]:
tile_database.columns

Index(['tile_name', 'chip_name', 'minx_polygon_pixels', 'miny_polygon_pixels',
       'maxx_polygon_pixels', 'maxy_polygon_pixels', 'polygon_vertices_pixels',
       'nw_corner_polygon_lat', 'nw_corner_polygon_lon',
       'se_corner_polygon_lat', 'se_corner_polygon_lon',
       'polygon_vertices_lon_lat', 'geometry', 'object_class', 'diameter (m)',
       'merged_bbox', 'bbox_within_inundation_bounds', 'Category1',
       'Category2', 'Category3', 'Category4', 'Category5', 'state'],
      dtype='object')

In [16]:
#https://localcoder.org/merge-the-bounding-boxes-near-by-into-one

,tile_name,minx_polygon_pixels,miny_polygon_pixels,maxx_polygon_pixels,maxy_polygon_pixels,nw_corner_polygon_lat,nw_corner_polygon_lon,se_corner_polygon_lat,se_corner_polygon_lon,geometry,object_class,diameter (m),merged_bbox,bbox_within_inundation_bounds,Category1,Category2,Category3,Category4,Category5,state
0,m_4007327_nw_18_060_20190809,974,314,1041,380,40.625753,-73.745466,40.625392,-73.744997,"POLYGON ((-73.74547 40.62575, -73.74547 40.625...",closed_roof_tank,39.6,True,False,0.0,0.0,0.0,0.0,0.0,New York
1,m_4007327_nw_18_060_20190809,1091,479,1157,512,40.624853,-73.744652,40.624669,-73.744188,"POLYGON ((-73.74465 40.62485, -73.74465 40.624...",closed_roof_tank,19.8,False,False,0.0,0.0,0.0,0.0,0.0,New York
2,m_4007327_nw_18_060_20190809,851,243,872,265,40.626147,-73.746331,40.626026,-73.746184,"POLYGON ((-73.74633 40.62615, -73.74633 40.626...",closed_roof_tank,12.6,False,False,0.0,0.0,0.0,0.0,0.0,New York
3,m_4007327_nw_18_060_20190809,843,284,897,335,40.625926,-73.746392,40.625646,-73.746014,"POLYGON ((-73.74639 40.62593, -73.74639 40.625...",closed_roof_tank,30.6,False,False,0.0,0.0,0.0,0.0,0.0,New York
4,m_4007327_nw_18_060_20190809,905,288,945,331,40.625899,-73.745952,40.625664,-73.745673,"POLYGON ((-73.74595 40.62590, -73.74595 40.625...",closed_roof_tank,24.0,False,False,0.0,0.0,0.0,0.0,0.0,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98164,m_3910409_se_13_060_20190803,2735,6826,2744,6836,39.777458,-104.920750,39.777404,-104.920687,"POLYGON ((-104.92075 39.77746, -104.92075 39.7...",narrow_closed_roof_tank,5.4,False,False,0.0,0.0,0.0,0.0,0.0,Colorado
98165,m_3910409_se_13_060_20190803,2748,6851,2756,6859,39.777323,-104.920659,39.777279,-104.920603,"POLYGON ((-104.92066 39.77732, -104.92066 39.7...",narrow_closed_roof_tank,4.8,False,False,0.0,0.0,0.0,0.0,0.0,Colorado
98166,m_3910409_se_13_060_20190803,2751,6778,2759,6784,39.777717,-104.920637,39.777685,-104.920581,"POLYGON ((-104.92064 39.77772, -104.92064 39.7...",narrow_closed_roof_tank,3.6,False,False,0.0,0.0,0.0,0.0,0.0,Colorado
98167,m_3910409_se_13_060_20190803,2750,6975,2758,6984,39.776652,-104.920645,39.776604,-104.920589,"POLYGON ((-104.92065 39.77665, -104.92065 39.7...",narrow_closed_roof_tank,4.8,False,False,0.0,0.0,0.0,0.0,0.0,Colorado


In [14]:
def write_gdf(gdf, output_filepath, output_filename = 'tile_level_annotations'):
    gdf.crs = "EPSG:4326" #assign projection

    #save geodatabase as json
    with open(os.path.join(output_filepath, output_filename+".json"), 'w') as file:
        file.write(gdf.to_json()) 

    ##save geodatabase as geojson 
    with open(os.path.join(output_filepath, output_filename+".geojson"), "w") as file:
        file.write(gdf.to_json()) 

    ##save geodatabase as shapefile
    gdf_shapefile = gdf.drop(columns=["chip_name","polygon_vertices_pixels","polygon_vertices_lon_lat"])
    gdf_shapefile.to_file(os.path.join(output_filepath,output_filename+".shp"))

# Save tile dabasebase

In [15]:
output_filepath = os.path.join(parent_directory,"complete_dataset","tile_level_annotations")
output_filename = 'tile_level_annotations'
write_gdf(tile_database, output_filepath, output_filename)

C:\Users\RAPIDU~1\AppData\Local\Temp/ipykernel_11084/3004279573.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_shapefile.to_file(os.path.join(output_filepath,output_filename+".shp"))
